# Что такое спуфинг?

Спуфинг (Spoofing) — это форма манипулирования рынком, при которой трейдер/маркет-мейкер/кит размещает крупные заявки (ордера) на покупку или продажу актива без намерения их исполнять. Цель спуфера — создать ложное впечатление о высоком спросе или предложении на определенном ценовом уровне, чтобы побудить других участников рынка совершать сделки, выгодные для манипулятора

# Как происходит манипуляция ценой:

* ### Для повышения цены: Спуфер ставит большой ордер на покупку (фальшивую стенку поддержки). Другие трейдеры, видя это, начинают покупать, опасаясь упустить рост, или ставят свои ордера выше спуфера. Цена начинает расти.
* ### Для понижения цены: Спуфер ставит большой ордер на продажу (фальшивую стенку сопротивления). Другие трейдеры, опасаясь падения, начинают продавать или ставить свои ордера ниже спуфера. Цена начинает падать.

# Как определить спуфинг через Time-weighted Order Flow Imbalance (OFI)?

Order Flow Imbalance (OFI) или дисбаланс потока ордеров — это метрика, которая измеряет разницу между давлением покупателей и продавцов в стакане заявок. Time-weighted OFI (взвешенный по времени OFI) придает больший вес более свежим или более значимым (в зависимости от реализации) ордерам.

# Обнаружение аномалий:

* ### Если на стороне покупки появляется очень большой ордер (потенциальный спуфинг на покупку), OFI резко возрастает (становится сильно положительным).
* ### Если на стороне продажи появляется очень большой ордер (потенциальный спуфинг на продажу), OFI резко падает (становится сильно отрицательным).

# К коду и формуле

## 1. Time-weighted Order Flow Imbalance (OFI)

OFI = bid_sum - ask_sum

OFI = Σ (quantity_bid_i * decay_factor^i) - Σ (quantity_ask_j * decay_factor^j), где

* quantity_bid_i: объем i-го ордера на покупку (bid)
* quantity_ask_j: объем j-го ордера на продажу (ask)
* decay_factor: фактор затухания (в коде 0.9)
* i: позиция (индекс) ордера на покупку в стакане (начиная с 0 для лучшего bid)
* j: позиция (индекс) ордера на продажу в стакане (начиная с 0 для лучшего ask)

OFI представляет собой чистый взвешенный объем. Положительное значение указывает на преобладание давления покупателей, а отрицательное — на преобладание давления продавцов, учитывая глубину рынка и "свежесть" ордеров (близость к лучшей цене)


### Что такое decay_factor = 0.9 
#### Параметр decay_factor (фактор затухания) в функции calculate_time_weighted_ofi используется для придания большего веса ордерам, находящимся ближе к лучшей цене (top of the book), и меньшего веса ордерам, находящимся дальше вглубь стакана

* #### Значение 0.9 означает, что каждый следующий уровень глубины стакана будет иметь 90% веса от предыдущего.
* #### Ордера на лучшей цене (индекс i=0) получают вес 0.9^0 = 1
* #### Ордера на следующем уровне (индекс i=1) получают вес 0.9^1 = 0.9
* #### Ордера на третьем уровне (индекс i=2) получают вес 0.9^2 = 0.81

Это позволяет OFI быть более чувствительным к изменениям в ближайших к рынку ордерах, которые с большей вероятностью повлияют на цену в краткосрочной перспективе


### Что такое i?
Переменная i является индексом элемента (ордера) в списке ордеров на покупку (bids) или продажу (asks)


## 2. Z-score 
Z-score — это мера того, насколько данное значение отклоняется от среднего значения набора данных, выраженная в единицах стандартного отклонения. В данном коде Z-score используется для обнаружения аномальных значений OFI, которые могут указывать на спуфинг

Z = |(X - μ) / σ|, где:

* X: текущее значение OFI (ofi_value)
* μ (мю): среднее значение OFI за определенный период (окно) (mean в коде)
* σ (сигма): стандартное отклонение значений OFI за тот же период (std в коде)

* ### Выбирается окно последних значений OFI:
window = self.ofi_values[-self.window_size:]
* ### Рассчитывается среднее значение (mean) OFI в этом окне:
mean = np.mean(window)
* ### Рассчитывается стандартное отклонение (std) OFI в этом окне:
std = np.std(window)
* ### Рассчитывается Z-score:
z_score = abs((ofi_value - mean) / std)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import datetime
from binance.client import Client
from binance.exceptions import BinanceAPIException
import warnings
warnings.filterwarnings('ignore')

# Инициализация клиента Binance с вашими API ключами
api_key = ''
api_secret = ''

client = Client(api_key, api_secret)

class SpufingDetector:
    def __init__(self, symbol='BTCUSDT', window_size=20, threshold=0.7, interval='1m'):
        """
        Инициализация детектора спуфинга
        
        Параметры:
        symbol (str): Торговая пара для анализа
        window_size (int): Размер окна для расчета OFI
        threshold (float): Пороговое значение для обнаружения аномалий
        interval (str): Интервал времени для анализа
        """
        self.symbol = symbol
        self.window_size = window_size
        self.threshold = threshold
        self.interval = interval
        self.ofi_values = []
        self.timestamps = []
        self.alerts = []
        
    def get_orderbook(self):
        """Получение данных ордербука"""
        try:
            depth = client.get_order_book(symbol=self.symbol, limit=20)
            return depth
        except BinanceAPIException as e:
            print(f"Ошибка API Binance: {e}")
            return None
    
    def calculate_time_weighted_ofi(self, orderbook, decay_factor=0.9):
        """
        Расчет Time-weighted Order Flow Imbalance
        
        Параметры:
        orderbook (dict): Данные ордербука
        decay_factor (float): Фактор затухания для придания большего веса свежим ордерам
        
        Возвращает:
        float: Значение OFI
        """
        if not orderbook:
            return 0
        
        current_time = time.time()
        
        # Извлечение ордеров на покупку и продажу
        bids = orderbook['bids']  # [[цена, объем], ...]
        asks = orderbook['asks']  # [[цена, объем], ...]
        
        # Расчет суммы взвешенных объемов для ордеров на покупку
        bid_sum = 0
        for i, (price, quantity) in enumerate(bids):
            # Вес зависит от позиции в стакане и времени
            time_weight = decay_factor ** i
            bid_sum += float(quantity) * time_weight
        
        # Расчет суммы взвешенных объемов для ордеров на продажу
        ask_sum = 0
        for i, (price, quantity) in enumerate(asks):
            # Вес зависит от позиции в стакане и времени
            time_weight = decay_factor ** i
            ask_sum += float(quantity) * time_weight
        
        # Расчет OFI как разницы между взвешенными объемами покупки и продажи
        ofi = bid_sum - ask_sum
        
        return ofi
    
    def detect_spoofing(self, ofi_value):
        """
        Обнаружение спуфинга на основе аномальных значений OFI
        
        Параметры:
        ofi_value (float): Текущее значение OFI
        
        Возвращает:
        bool: True, если обнаружен спуфинг, иначе False
        """
        if len(self.ofi_values) < self.window_size:
            return False
        
        # Расчет Z-показателя для текущего значения OFI
        window = self.ofi_values[-self.window_size:]
        mean = np.mean(window)
        std = np.std(window)
        
        if std == 0:
            return False
        
        z_score = abs((ofi_value - mean) / std)
        
        # Обнаружение аномалии, если Z-показатель превышает пороговое значение
        is_anomaly = z_score > self.threshold
        
        if is_anomaly:
            # Определение направления манипуляции
            direction = "ПОКУПКА" if ofi_value > 0 else "ПРОДАЖА"
            self.alerts.append({
                'timestamp': datetime.datetime.now(),
                'z_score': z_score,
                'ofi': ofi_value,
                'direction': direction
            })
            
        return is_anomaly
    
    def run_analysis(self, duration_minutes=60):
        """
        Запуск анализа в реальном времени
        
        Параметры:
        duration_minutes (int): Продолжительность анализа в минутах
        """
        print(f"Запуск анализа спуфинга для {self.symbol} на {duration_minutes} минут...")
        
        start_time = time.time()
        end_time = start_time + (duration_minutes * 60)
        
        try:
            while time.time() < end_time:
                # Получение данных ордербука
                orderbook = self.get_orderbook()
                
                if orderbook:
                    # Расчет OFI
                    ofi_value = self.calculate_time_weighted_ofi(orderbook)
                    current_time = datetime.datetime.now()
                    
                    # Сохранение значений для анализа
                    self.ofi_values.append(ofi_value)
                    self.timestamps.append(current_time)
                    
                    # Обнаружение спуфинга
                    is_spoofing = self.detect_spoofing(ofi_value)
                    
                    if is_spoofing:
                        print(f"[{current_time}] ОБНАРУЖЕНА МАНИПУЛЯЦИЯ: {self.alerts[-1]['direction']} (OFI: {ofi_value:.2f}, Z-показатель: {self.alerts[-1]['z_score']:.2f})")
                    
                    # Если накоплено достаточно данных, можно визуализировать
                    if len(self.ofi_values) % 30 == 0:
                        self.visualize_ofi()
                
                # Пауза между запросами, чтобы не превысить лимиты API
                time.sleep(2)
                
        except KeyboardInterrupt:
            print("Анализ остановлен пользователем.")
        finally:
            print("Анализ завершен.")
            self.generate_report()
    
    def visualize_ofi(self):
        """Визуализация значений OFI"""
        plt.figure(figsize=(12, 6))
        
        # График OFI
        plt.subplot(2, 1, 1)
        plt.plot(self.timestamps, self.ofi_values, label='OFI')
        plt.title(f'Time-weighted Order Flow Imbalance для {self.symbol}')
        plt.xlabel('Время')
        plt.ylabel('OFI')
        plt.legend()
        plt.grid(True)
        
        # Распределение значений OFI
        plt.subplot(2, 1, 2)
        plt.hist(self.ofi_values, bins=30, alpha=0.7)
        plt.title('Распределение значений OFI')
        plt.xlabel('OFI')
        plt.ylabel('Частота')
        plt.grid(True)
        
        plt.tight_layout()
        plt.savefig(f'ofi_analysis_{self.symbol}_{datetime.datetime.now().strftime("%Y%m%d_%H%M%S")}.png')
        plt.close()
    
    def generate_report(self):
        """Генерация отчета о обнаруженных манипуляциях"""
        if not self.alerts:
            print("Манипуляции не обнаружены.")
            return
        
        print("\n=== ОТЧЕТ О МАНИПУЛЯЦИЯХ СТАКАНА ===")
        print(f"Торговая пара: {self.symbol}")
        print(f"Период анализа: {self.timestamps[0]} - {self.timestamps[-1]}")
        print(f"Количество обнаруженных манипуляций: {len(self.alerts)}")
        print(f"Пороговое значение Z-показателя: {self.threshold}")
        
        print("\nТОП-5 наиболее значимых манипуляций:")
        sorted_alerts = sorted(self.alerts, key=lambda x: x['z_score'], reverse=True)
        
        for i, alert in enumerate(sorted_alerts[:5], 1):
            print(f"{i}. [{alert['timestamp']}] Направление: {alert['direction']}, OFI: {alert['ofi']:.2f}, Z-показатель: {alert['z_score']:.2f}")
        
        # Сохранение всех оповещений в CSV
        if self.alerts:
            df_alerts = pd.DataFrame(self.alerts)
            filename = f'spoofing_alerts_{self.symbol}_{datetime.datetime.now().strftime("%Y%m%d_%H%M%S")}.csv'
            df_alerts.to_csv(filename, index=False)
            print(f"\nПолный отчет сохранен в файл: {filename}")
        
        # Финальная визуализация
        self.visualize_ofi()
        print(f"Визуализация сохранена в файл: ofi_analysis_{self.symbol}_*.png")

# Пример использования
if __name__ == "__main__":
    # Настройка параметров анализа
    symbol = input("Введите торговую пару (например, BTCUSDT): ") or "BTCUSDT"
    window_size = int(input("Введите размер окна для расчета (например, 20): ") or "20")
    threshold = float(input("Введите пороговое значение Z-показателя (например, 2.5): ") or "2.5")
    duration = int(input("Введите продолжительность анализа в минутах (например, 60): ") or "60")
    
    # Создание и запуск детектора спуфинга
    detector = SpufingDetector(symbol=symbol, window_size=window_size, threshold=threshold)
    detector.run_analysis(duration_minutes=duration)


# Расширенная версия для ретроспективного анализа исторических данных
def analyze_historical_data(symbol='BTCUSDT', start_time=None, end_time=None, interval='1m'):
    """
    Анализ исторических данных для поиска спуфинга
    
    Параметры:
    symbol (str): Торговая пара
    start_time (str): Начальное время в формате 'YYYY-MM-DD HH:MM:SS'
    end_time (str): Конечное время в формате 'YYYY-MM-DD HH:MM:SS'
    interval (str): Интервал времени
    """
    if start_time is None:
        # По умолчанию - последние 24 часа
        start_time = (datetime.datetime.now() - datetime.timedelta(days=1)).strftime('%Y-%m-%d %H:%M:%S')
    
    if end_time is None:
        end_time = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    
    print(f"Анализ исторических данных для {symbol} с {start_time} по {end_time}...")
    
    # Преобразование времени в миллисекунды
    start_ms = int(datetime.datetime.strptime(start_time, '%Y-%m-%d %H:%M:%S').timestamp() * 1000)
    end_ms = int(datetime.datetime.strptime(end_time, '%Y-%m-%d %H:%M:%S').timestamp() * 1000)
    
    # Получение исторических данных
    try:
        # Получение свечей
        klines = client.get_klines(
            symbol=symbol,
            interval=interval,
            startTime=start_ms,
            endTime=end_ms
        )
        
        # Преобразование данных в DataFrame
        df = pd.DataFrame(klines, columns=[
            'timestamp', 'open', 'high', 'low', 'close', 'volume',
            'close_time', 'quote_asset_volume', 'number_of_trades',
            'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'
        ])
        
        # Преобразование типов данных
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        for col in ['open', 'high', 'low', 'close', 'volume']:
            df[col] = df[col].astype(float)
        
        # Расчет показателей
        df['price_change'] = df['close'].diff()
        df['volume_change'] = df['volume'].diff()
        
        # Расчет приблизительного OFI на основе истории сделок
        # Вместо реальных данных ордербука используем соотношение покупок/продаж
        df['buy_volume'] = df['taker_buy_base_asset_volume'].astype(float)
        df['sell_volume'] = df['volume'] - df['buy_volume']
        df['ofi_proxy'] = df['buy_volume'] - df['sell_volume']
        
        # Расчет Z-показателя для обнаружения аномалий
        window_size = 20
        df['ofi_rolling_mean'] = df['ofi_proxy'].rolling(window=window_size).mean()
        df['ofi_rolling_std'] = df['ofi_proxy'].rolling(window=window_size).std()
        df['z_score'] = (df['ofi_proxy'] - df['ofi_rolling_mean']) / df['ofi_rolling_std']
        
        # Обнаружение потенциальных манипуляций
        threshold = 2.5
        df['potential_spoofing'] = np.abs(df['z_score']) > threshold
        
        # Визуализация результатов
        plt.figure(figsize=(14, 10))
        
        # График цены
        plt.subplot(3, 1, 1)
        plt.plot(df['timestamp'], df['close'], label='Цена закрытия')
        plt.title(f'Анализ потенциального спуфинга для {symbol}')
        plt.ylabel('Цена')
        plt.legend()
        plt.grid(True)
        
        # График OFI
        plt.subplot(3, 1, 2)
        plt.plot(df['timestamp'], df['ofi_proxy'], label='OFI (прокси)')
        plt.ylabel('OFI')
        plt.legend()
        plt.grid(True)
        
        # График Z-показателя
        plt.subplot(3, 1, 3)
        plt.plot(df['timestamp'], df['z_score'], label='Z-показатель')
        plt.axhline(y=threshold, color='r', linestyle='--', label=f'Порог {threshold}')
        plt.axhline(y=-threshold, color='r', linestyle='--')
        # Выделение потенциальных манипуляций
        spoofing_points = df[df['potential_spoofing']]
        plt.scatter(spoofing_points['timestamp'], spoofing_points['z_score'], 
                   color='red', s=50, label='Потенциальный спуфинг')
        plt.xlabel('Время')
        plt.ylabel('Z-показатель')
        plt.legend()
        plt.grid(True)
        
        plt.tight_layout()
        filename = f'historical_spoofing_analysis_{symbol}_{datetime.datetime.now().strftime("%Y%m%d_%H%M%S")}.png'
        plt.savefig(filename)
        plt.close()
        
        # Сохранение результатов в CSV
        df_results = df[df['potential_spoofing']].copy()
        if not df_results.empty:
            # Добавление направления манипуляции
            df_results['direction'] = df_results['ofi_proxy'].apply(lambda x: "ПОКУПКА" if x > 0 else "ПРОДАЖА")
            # Сохранение только важных колонок
            df_results = df_results[['timestamp', 'close', 'ofi_proxy', 'z_score', 'direction']]
            csv_filename = f'historical_spoofing_results_{symbol}_{datetime.datetime.now().strftime("%Y%m%d_%H%M%S")}.csv'
            df_results.to_csv(csv_filename, index=False)
            
            print(f"Обнаружено {len(df_results)} потенциальных манипуляций.")
            print(f"Визуализация сохранена в файл: {filename}")
            print(f"Результаты сохранены в файл: {csv_filename}")
        else:
            print("Потенциальные манипуляции не обнаружены.")
        
        return df
        
    except BinanceAPIException as e:
        print(f"Ошибка API Binance: {e}")
        return None

# Пример использования анализа исторических данных
if __name__ == "__main__" and False:  # Измените на True для запуска
    symbol = input("Введите торговую пару для исторического анализа (например, BTCUSDT): ") or "BTCUSDT"
    start_time = input("Введите начальное время (YYYY-MM-DD HH:MM:SS) или оставьте пустым для 24 часов назад: ")
    end_time = input("Введите конечное время (YYYY-MM-DD HH:MM:SS) или оставьте пустым для текущего времени: ")
    
    if not start_time:
        start_time = None
    if not end_time:
        end_time = None
    
    df = analyze_historical_data(symbol=symbol, start_time=start_time, end_time=end_time)

Введите торговую пару (например, BTCUSDT):  BNBUSDT
Введите размер окна для расчета (например, 20):  20
Введите пороговое значение Z-показателя (например, 2.5):  3
Введите продолжительность анализа в минутах (например, 60):  60


Запуск анализа спуфинга для BNBUSDT на 60 минут...
[2025-05-22 13:14:21.087661] ОБНАРУЖЕНА МАНИПУЛЯЦИЯ: ПОКУПКА (OFI: 206.26, Z-показатель: 3.05)
[2025-05-22 13:14:27.988354] ОБНАРУЖЕНА МАНИПУЛЯЦИЯ: ПРОДАЖА (OFI: -85.97, Z-показатель: 3.17)
[2025-05-22 13:15:16.785422] ОБНАРУЖЕНА МАНИПУЛЯЦИЯ: ПОКУПКА (OFI: 636.08, Z-показатель: 3.76)
[2025-05-22 13:18:17.007377] ОБНАРУЖЕНА МАНИПУЛЯЦИЯ: ПОКУПКА (OFI: 166.40, Z-показатель: 3.20)
[2025-05-22 13:19:11.027463] ОБНАРУЖЕНА МАНИПУЛЯЦИЯ: ПОКУПКА (OFI: 647.15, Z-показатель: 3.39)
[2025-05-22 13:19:54.673161] ОБНАРУЖЕНА МАНИПУЛЯЦИЯ: ПОКУПКА (OFI: 120.77, Z-показатель: 3.65)
[2025-05-22 13:24:39.602310] ОБНАРУЖЕНА МАНИПУЛЯЦИЯ: ПОКУПКА (OFI: 73.37, Z-показатель: 3.21)


# Как интерпритировать результаты?
* Если OFI > 0, то это ПОКУПКА
* Если OFI < 0, то это ПРОДАЖА

### Z-показатель измеряет, насколько текущее значение OFI отклоняется от среднего значения OFI за предыдущий период (заданный window_size) в единицах стандартного отклонения

* Пороговое значение Z-показателя у вас установлено на 2 (threshold = 2). Скрипт сообщает о манипуляции, когда абсолютное значение Z-показателя превышает этот порог
* Z-показатель 3.08 означает, что текущий OFI (1967.4) на 3.08 стандартных отклонения выше, чем средний OFI за последние 20 измерений. Это статистически значимое отклонение, которое и вызвало срабатывание детектора. Чем выше Z-показатель, тем более "необычным" или "аномальным" является текущий дисбаланс по сравнению с недавней историей


* #### Манипуляция "ПОКУПКА" с высоким положительным OFI и Z-показателем > порога: Может указывать на попытку "накачать" цену вверх, создавая иллюзию сильного спроса
* #### Манипуляция "ПРОДАЖА" с высоким отрицательным OFI и Z-показателем > порога: Может указывать на попытку "обвалить" цену, создавая иллюзию сильного предложения


## Реакция на отмену ордера:

* Когда спуфер отменяет свой большой ордер, этот объем исчезает из стакана
* При следующем вызове get_orderbook() (в вашем коде это происходит через time.sleep(2) секунды + время выполнения запроса и расчетов)
* Соответственно, calculate_time_weighted_ofi рассчитает новое значение OFI, которое, скорее всего, вернется к "нормальному" уровню или даже качнется в противоположную сторону (если спуфер одновременно с отменой выставил ордер на другой стороне)